<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Persist, Deploy, and Score an SPSS Model to Predict Customer Churn</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
  <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/blob/master/spark/customer-satisfaction-prediction/images/users_banner_2-03.png?raw=true" width="600" alt="Icon"> </th>
   </tr>
</table>

This notebook demonstrates how to deploy an SPSS model and score test data. 

You will use the **Telco Customer Churn** data set which contains anonymous data about customers of a fictitious telecommunication company. Use the details of this data set to predict customer churn which is critical to business as it is easier to retain existing customers rather than acquire new ones.

Some familiarity with Python is helpful. This notebook is compatible with CP4D 3.0 and Python 3.6.


## Learning goals

In this notebook you learn how to:

-  Save an SPSS model.
-  Deploy the SPSS model online.
-  Use the deployed model to score data.


## Contents

1.	[Setting up](#setup)
2.	[Save the SPSS model](#save)
3.  [Deploy the SPSS model (with deployment space only)](#deploy)
4.	[Summary](#summary)

To get started on CP4D 3.0, find documentation on installation and set up <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/cpd/overview/welcome.html" target="_blank" rel="noopener no referrer">here</a>.

## 1. Setting up <a id="setup"></a>

Import the `watson-machine-learning-client` module.
<div class="alert alert-block alert-info">
For more information about the <b>Watson Machine Learning Python client (V4)</b>, please refer to the <a href="https://wml-api-pyclient-dev-v4.mybluemix.net/" target="_blank" rel="noopener no referrer">Python client documentation</a>. If you're using the notebook within a project on your CP4D cluster, you do not need to install this package as it comes pre-installed with the notebooks. The installation code below is for demonstration but is non-executable at this stage.
</div>

In [1]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

**Authenticate the Python client on CP4D.**

<div class="alert alert-block alert-info">To find your authentication information (your credentials) follow the steps provided here in the <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/ml-authentication-local.html" target="_blank" rel="noopener no referrer">Documentation.</a></div>

In [2]:
# Enter your credentials here.

from project_lib.utils import environment
url = environment.get_common_api_url()

import sys,os,os.path
token = os.environ['USER_ACCESS_TOKEN']

wml_credentials = {
     "instance_id": "openshift",
     "token": token,
     "url": url,
     "version": "3.0.0"
}

Now, instantiate a `WatsonMachineLearningAPIClient` object.

In [3]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [4]:
client.version

'1.0.267'

<div class="alert alert-block alert-info">
You have a choice to either save the model in the <b>project</b> or the <b>deployment space</b>:<br><br>
    <li> If you're saving the model in your project, you have to set the default project using the python client.</li><br>
    <li>If you're saving the model in the deployment space, you have to obtain the space UID of the deployment space you've created. Then you'd use this to set the default space using the python client. From there you'll be able to deploy and score the model in your deployment space.</li></div>

### To set the default project, use the following code.

In [5]:
from project_lib import Project
project = Project.access()
project_id = project.get_metadata()["metadata"]["guid"]

client.set.default_project(project_id)

'SUCCESS'

### To set the default space, follow these steps.

<div class="alert alert-block alert-info">
You can create your own <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/ml-spaces_local.html" target="_blank" rel="noopener no referrer">deployment space</a> by selecting <b>Analytics deployments</b> under <b>Analyze</b> from the Navigation Menu on the top left of this page.</div>

Alternatively, you can create a deployment and obtain its UID using the code in the following cell. The cell is not executable cell at this stage, but you can enter the name of your space in the metadata and use it if needed.

In [12]:
# Obtain the UId of your space
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['guid'])

**Action:** Enter the name of your deployment space in the code below: `space_uid = guid_from_space_name(client, 'YOUR DEPLOYMENT SPACE')`.

In [13]:
# Enter the name of your deployment space here:
space_uid = guid_from_space_name(client, 'YOUR DEPLOYMENT SPACE')
print("Space UID = " + space_uid)

Space UID = df813339-652b-41fb-ba4a-46980ed101a2


You can set the default space using the cell below.

In [14]:
client.set.default_space(space_uid)

Unsetting the project_id ...


'SUCCESS'

<a id="save"></a>
## 2. Save the SPSS model

Download the SPSS sample model from the <a href="https://github.com/pmservice/wml-sample-models" target="_blank" rel="noopener no referrer">Git repository</a>.

<div class="alert alert-block alert-info">
<b>Note:</b> You may need to install the <tt>wget</tt> package. To install the <tt>wget</tt> package, run the following command.</div>

In [6]:
!pip install --upgrade wget

Requirement already up-to-date: wget in /opt/conda/envs/Python-3.6-WMLCE/lib/python3.6/site-packages (3.2)


In [7]:
# Download the sample SPSS model.
import os
import wget

sample_dir = 'spss_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)

filename=os.path.join(sample_dir, 'customer-satisfaction-prediction.str')
if not os.path.isfile(filename):
    filename = wget.download('https://github.com/pmservice/wml-sample-models/raw/master/spss/customer-satisfaction-prediction/model/customer-satisfaction-prediction.str',\
                             out=sample_dir)
print(filename)

spss_sample_model/customer-satisfaction-prediction.str


Save the downloaded SPSS sample model as *SPSS model for Churn prediction*. First, you need to create the model metadata.

In [8]:
# Save the SPSS model to the WML repository.
# Model Metadata.
software_spec_uid = client.software_specifications.get_uid_by_name('spss-modeler_18.1')

meta_props={
    client.repository.ModelMetaNames.NAME: "SPSS model for Churn prediction",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
    client.repository.ModelMetaNames.TYPE: "spss-modeler_18.1"
}

<div class="alert alert-block alert-info">To list the supported software specifications, run <tt>client.software_specifications.list()</tt>.<br>To find more information about the frameworks with their respective <b>Types</b> and <b>Software Specifications</b>, visit the <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/wmls/wmls-deploy-python-types.html" target="_blank" rel="noopener no referrer">documentation</a>.</div>

You can extract the model UID from the saved model details.

In [15]:
# Create the model artifact.
model_artifact = client.repository.store_model(filename, meta_props=meta_props)
model_uid = client.repository.get_model_uid(model_artifact)
print("Model UID = " + model_uid)

Model UID = 41af3180-3d1b-4fdf-ae8c-4d3b3f2e210a


Get the saved model metadata using the model UID.

In [16]:
# Details about the model.
model_details = client.repository.get_details(model_uid)
from pprint import pprint
pprint(model_details)

{'entity': {'content_status': {'state': 'persisted'},
            'name': 'SPSS model for Churn prediction',
            'software_spec': {'id': '5c3cad7e-507f-4b2a-a9a3-ab53a21dee8b'},
            'space': {'href': '/v4/spaces/df813339-652b-41fb-ba4a-46980ed101a2',
                      'id': 'df813339-652b-41fb-ba4a-46980ed101a2'},
            'type': 'spss-modeler_18.1'},
 'metadata': {'created_at': '2020-03-13T16:27:50.002Z',
              'guid': '41af3180-3d1b-4fdf-ae8c-4d3b3f2e210a',
              'href': '/v4/models/41af3180-3d1b-4fdf-ae8c-4d3b3f2e210a?space_id=df813339-652b-41fb-ba4a-46980ed101a2',
              'id': '41af3180-3d1b-4fdf-ae8c-4d3b3f2e210a',
              'modified_at': '2020-03-13T16:27:53.002Z',
              'name': 'SPSS model for Churn prediction',
              'owner': '1000330999',
              'space_id': 'df813339-652b-41fb-ba4a-46980ed101a2'}}


You can list all stored models using the `list_models` method.

In [17]:
# Display a list of all the models.
client.repository.list_models()

------------------------------------  -------------------------------  ------------------------  -----------------
GUID                                  NAME                             CREATED                   TYPE
41af3180-3d1b-4fdf-ae8c-4d3b3f2e210a  SPSS model for Churn prediction  2020-03-13T16:27:50.002Z  spss-modeler_18.1
------------------------------------  -------------------------------  ------------------------  -----------------


<div class="alert alert-block alert-info">
From the list of stored models, you can see that model is successfully saved.<br><br>
If you've set the default project, this means you've saved the model in your project. You can see the saved model in your project UI by clicking on your project name in the breadcrumb at the top of the application.<br><br>
If you've set the default space, this means that you've saved the model in your deployment space. You can view your model by selecting <b>Analytics Deployments</b> under <b>Analyze</b> from the Navigation Menu and clicking on your deployment space name.</div>

If you're using a deployment space, proceed to Section 3: [Deploy the SPSS Model (with deployment space only)](#deploy). If not, you may skip to the [summary](#summary).

## 3. Deploy the SPSS Model (with deployment space only) <a id="deploy"></a>

Using the code below to create an online deployment using the model UID obtained in the previous section. Create the deployment metadata using the code below.

In [18]:
# Deployment metadata.
deploy_meta = {
    client.deployments.ConfigurationMetaNames.NAME: "Sample SPSS model deployment",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [19]:
# Create the deployment.
deployment_details = client.deployments.create(model_uid, meta_props=deploy_meta)



#######################################################################################

Synchronous deployment creation for uid: '41af3180-3d1b-4fdf-ae8c-4d3b3f2e210a' started

#######################################################################################


initializing.........
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='6bab756f-3d00-4afd-9c6e-4d8468853270'
------------------------------------------------------------------------------------------------




Get the list of all deployments in the deployment space.

In [20]:
# List the deployments.
client.deployments.list()

------------------------------------  ----------------------------  -----  ------------------------  -------------
GUID                                  NAME                          STATE  CREATED                   ARTIFACT_TYPE
6bab756f-3d00-4afd-9c6e-4d8468853270  Sample SPSS model deployment  ready  2020-03-13T16:27:58.394Z  model
------------------------------------  ----------------------------  -----  ------------------------  -------------


<div class="alert alert-block alert-info">
From the list of deployed models, you can see that model was successfully deployed in the deployment space.</div>

Now, you can check details of your deployments.

In [21]:
# Deployment UID.
deployment_uid = client.deployments.get_uid(deployment_details)
print('Deployment uid = {}'.format(deployment_uid))

Deployment uid = 6bab756f-3d00-4afd-9c6e-4d8468853270


In [22]:
# Deployment details.
print(client.deployments.get_details(deployment_uid))

{'metadata': {'parent': {'href': ''}, 'name': 'Sample SPSS model deployment', 'guid': '6bab756f-3d00-4afd-9c6e-4d8468853270', 'description': '', 'id': '6bab756f-3d00-4afd-9c6e-4d8468853270', 'modified_at': '2020-03-13T16:27:58.394Z', 'created_at': '2020-03-13T16:27:58.394Z', 'href': '/v4/deployments/6bab756f-3d00-4afd-9c6e-4d8468853270', 'space_id': 'df813339-652b-41fb-ba4a-46980ed101a2'}, 'entity': {'name': 'Sample SPSS model deployment', 'custom': {}, 'online': {}, 'description': '', 'space': {'id': 'df813339-652b-41fb-ba4a-46980ed101a2', 'href': '/v4/spaces/df813339-652b-41fb-ba4a-46980ed101a2'}, 'status': {'state': 'ready', 'online_url': {'url': 'https://internal-nginx-svc:12443/v4/deployments/6bab756f-3d00-4afd-9c6e-4d8468853270/predictions'}}, 'asset': {'id': '41af3180-3d1b-4fdf-ae8c-4d3b3f2e210a', 'href': '/v4/models/41af3180-3d1b-4fdf-ae8c-4d3b3f2e210a?space_id=df813339-652b-41fb-ba4a-46980ed101a2'}, 'space_id': 'df813339-652b-41fb-ba4a-46980ed101a2'}}


Prepare the scoring payload with values to score against the deployed model.

In [23]:
# Prepare scoring payload.
job_payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [{
        'fields':['customerID','gender','SeniorCitizen','Partner','Dependents','tenure','PhoneService','MultipleLines','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','Contract','PaperlessBilling','PaymentMethod','MonthlyCharges','TotalCharges','Churn','SampleWeight'],
        'values':[['3638-WEABW','Female',0,'Yes','No',58,'Yes','Yes','DSL','No','Yes','No','Yes','No','No','Two year','Yes','Credit card (automatic)',59.9,3505.1,'No',2.768]]
    }]
}
pprint(job_payload)

{'input_data': [{'fields': ['customerID',
                            'gender',
                            'SeniorCitizen',
                            'Partner',
                            'Dependents',
                            'tenure',
                            'PhoneService',
                            'MultipleLines',
                            'InternetService',
                            'OnlineSecurity',
                            'OnlineBackup',
                            'DeviceProtection',
                            'TechSupport',
                            'StreamingTV',
                            'StreamingMovies',
                            'Contract',
                            'PaperlessBilling',
                            'PaymentMethod',
                            'MonthlyCharges',
                            'TotalCharges',
                            'Churn',
                            'SampleWeight'],
                 'values': [['3638-WEABW',
 

Use the client `score` method to score test data about a customer.

In [24]:
# Perform prediction and display the result.
job_details = client.deployments.score(deployment_uid, job_payload)
pprint(job_details)

{'predictions': [{'fields': ['customerID',
                             'Churn',
                             'Predicted Churn',
                             'Probability of Churn'],
                  'values': [['3638-WEABW', 'No', 'No', 0.0526309571556145]]}]}


As you can see, this sample telco customer is satisfied ("Predicted Churn", "No").

<a id="summary"></a>
## 4. Summary     

You successfully completed this notebook! 

You learned how to use Watson Machine Learning to create and deploy an SPSS model. 

### Resources <a id="resources"></a>

To learn more about configurations used in this notebook or more sample notebooks, tutorials, documentation, how-tos, and blog posts, check out these links:

<div class="alert alert-block alert-success"><a id="resources"></a>

<h4>IBM documentation</h4>
<br>
 <li> <a href="https://wml-api-pyclient-dev-v4.mybluemix.net" target="_blank" rel="noopener no referrer">watson-machine-learning</a></li> 
 <li> <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/cpd/overview/welcome.html" target="_blank" rel="noopener noreferrer">CP4D 3.0</a></li>
 
<h4> IBM Samples</h4>
<br>
 <li> <a href="https://github.com/IBMDataScience/sample-notebooks" target="_blank" rel="noopener noreferrer">Sample notebooks</a></li>
 
<h4> Others</h4>
<br>
 <li> <a href="https://www.python.org" target="_blank" rel="noopener noreferrer">Official Python website</a></li>
 <li> <a href="https://matplotlib.org" target="_blank" rel="noopener noreferrer">Matplotlib: Python plotting</a></li>
 <li> <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html" target="_blank" rel="noopener noreferrer">scikit-learn: Grid Search</a></li>
 </div>

### Authors

**Lukasz Cmielowski**, Ph.D., is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increase the clients' ability to turn data into actionable knowledge.  
**Jihyoung Kim**, Ph.D., is a Data Scientist at IBM who strives to make data science easy for everyone through Watson Studio.

Copyright © 2017-2020 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>